In [53]:
import pandas as pd
import numpy as np
import nltk
import re

In [54]:
# !pip install nltk

In [55]:
# 5+6
# !pip install spacy

In [56]:
import nltk
nltk.__version__

'3.8.1'

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
import string
import spacy



In [58]:
# df = pd.read_csv("training.csv", encoding="ISO-8859-1")
df = pd.read_csv("sentiment140_with_location.csv", header=None, names=["target", "unique_id", "date", "flag", "user_id", "text","location"], encoding="ISO-8859-1")
df.sample(10)

,target,unique_id,date,flag,user_id,text,location
1289478,4,2002619049,Tue Jun 02 04:43:34 PDT 2009,NO_QUERY,marapole90,watched &quot; The house bunny and Changeling&...,Dominican Republic
1449567,4,2062853769,Sun Jun 07 00:31:18 PDT 2009,NO_QUERY,EliasKeppens,"@macrystal same , kinda woke up early tho",Germany
509246,0,2189359607,Mon Jun 15 23:51:09 PDT 2009,NO_QUERY,HeatherHavoc,I am wracking my brain trying to figure out WH...,Mexico
1455093,4,2063382539,Sun Jun 07 02:26:06 PDT 2009,NO_QUERY,LoveMyNuts,"(@digall) No not letting my nuts out, freezing...",Cuba
517561,0,2191325132,Tue Jun 16 05:02:06 PDT 2009,NO_QUERY,courtneycurran,"@cdavis620 I'm working all day, too how was y...",Colombia
510629,0,2189700566,Tue Jun 16 00:40:54 PDT 2009,NO_QUERY,Bre_Rose,bah - 8 hour days a work suck - especially whe...,South Africa
1267244,4,1999804861,Mon Jun 01 20:51:15 PDT 2009,NO_QUERY,LilliJ,@GenaLivings you warmed my heart today..,Japan
1338543,4,2018221235,Wed Jun 03 09:37:39 PDT 2009,NO_QUERY,patriciaco,Mar Roxas has a Twitter account. Awesome.,Chile
666252,0,2245220125,Fri Jun 19 15:49:34 PDT 2009,NO_QUERY,cerebralhypoxia,Unbelievable! Tied after down 7 runs! Poor Woo...,Argentina
1412004,4,2056574070,Sat Jun 06 11:23:41 PDT 2009,NO_QUERY,KelsMaine,had a moment. i love my Bff,Ecuador


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   target     1600000 non-null  int64 
 1   unique_id  1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   flag       1600000 non-null  object
 4   user_id    1600000 non-null  object
 5   text       1600000 non-null  object
 6   location   1600000 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [60]:
df.shape

(1600000, 7)

In [61]:
df.drop(['unique_id','date','flag','user_id'], axis = 1, inplace = True) 

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   target    1600000 non-null  int64 
 1   text      1600000 non-null  object
 2   location  1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


In [63]:
df.sample(10)

,target,text,location
625073,0,@ALKINGyouworms you don't like Drake?,Tanzania
307206,0,"I hate hayfever, has kept me up nearly all nite",Mexico
1068093,4,Off to the retreat... i am excited,Germany
1497721,4,@lonemat but i want to customize each ellies c...,Japan
518703,0,I hate school. I don't want to go to school tmr!,Egypt
874857,4,@photoshoptips That's amazing,Ethiopia
1490704,4,@adammshankman i live thirty minutes from atla...,Thailand
604219,0,@SpeedyRockett what's that for? math? or langu...,UK
863302,4,up and off to watch the finals of the Malta Ph...,Thailand
938926,4,@esmeeworld I just saw your video of Outta Her...,UK


In [64]:
df.rename(columns={'target': 'sentiment', 'text': 'tweet'}, inplace = True)

In [65]:
df.sample(10)

,sentiment,tweet,location
715685,0,@rreezy: LuCky!! I want some chinese food!!,Indonesia
1148168,4,@dougiemcfly I loved the show in Rio! &quot;Th...,Ecuador
1491703,4,watching the SAW IV commentary whilst folding ...,Italy
995354,4,Just watched jay leno show. Kings of leon were...,Australia
1437393,4,@Shaantastic i absolutly can not get enough o...,Russia
4701,0,"i miss cristÃ¯Â¿Â½bal, renata, alysson, lucas,...",Thailand
1497209,4,Bout 2 fix more nachos and punch 4 da game.,Ethiopia
644912,0,"@peech____ Now if people protest, they'll not...",Brazil
363960,0,stuck in traffic,Peru
207537,0,awww i love aiden,China


In [66]:
df = df.dropna()
df.sample(10)

,sentiment,tweet,location
821015,4,Now this is what I call a Saturday... God love...,Thailand
871325,4,Golfing with my mother,Mexico
356711,0,"said goodbye to seniors cried a SHIT load, at...",Thailand
1462353,4,@beckwaite yep 10am Spoon at Paddo tomorrow,Jamaica
42264,0,buuuuuuuuh!!!! I want to see JONAS tomorrow!!!...,Thailand
344132,0,@spr33 @ScottSharman @Rocks4Ever i hate that c...,Argentina
558882,0,@jadeepops why?,Indonesia
1528966,4,@Thebombshell okie doke bestie!,Jamaica
1225322,4,love my new bg. hooorah.,Egypt
1183480,4,Finally home,Kenya


In [67]:
print(df.info())
print("The Shape Of The Dataset Is :",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   tweet      1600000 non-null  object
 2   location   1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB
None
The Shape Of The Dataset Is : (1600000, 3)


In [68]:
df['sentiment'] = df['sentiment'].replace({0: 'negative', 4: 'positive'})

In [69]:
df.sample(5)

,sentiment,tweet,location
1337022,positive,@lulubunsexxy thanx lulu... It wasnt an easy r...,Antarctica
691889,negative,@MaryBoomBoom Maria where have u been lately??...,Japan
1047386,positive,@TheLastDoctor One problem with that. https://...,Canada
361112,negative,omg. i never posted my video last night. im a ...,Brazil
200611,negative,What a beautiful day... to be working,Kenya


In [70]:
# Remove URLs, special characters, punctuation, and numbers from tweet column
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[0-9]+', '', x))


In [71]:
print(df.sample(10))
print('\033[1m' + 'The information of the dataset is :' + '\033[0m')
print(df.info())
print('\033[1m' + 'The Shape Of The Dataset Is :' + '\033[0m',df.shape)

        sentiment                                              tweet  \
841091   positive                  I got invited to go to the beach    
453364   negative  Is finally home and resting and is really miss...   
1271298  positive  honorsociety i love you guys  cant wait for th...   
157095   negative  MsCouture LOL Its Kind of Catchy But Yeah He M...   
992615   positive  chelsealinn just be grateful you didnt have a ...   
136640   negative                             stress por accesorios    
1348460  positive  DaniArtaud Im sure when you play SF Asia will ...   
45661    negative  cariadweez Thats good  and thanks Weez  will l...   
1377457  positive               dfizzy i hope you feel better damon    
91556    negative   Wildcatcdc OMG Id cry   How was your Mothers Day   

          location  
841091      Russia  
453364       Spain  
1271298   Thailand  
157095     Ecuador  
992615   Australia  
136640   Indonesia  
1348460  Argentina  
45661    Australia  
1377457    Germany

In [72]:
df['tweet'] = df['tweet'].str.lower()

In [73]:
df.sample(10)

,sentiment,tweet,location
607749,negative,i looked at the weather today and it said amp...,Canada
490478,negative,get the fuck out of my fandom bitches,Egypt
1132927,positive,my new sweety just came over with his cousin a...,Colombia
896192,positive,bartmillard have u guys done bohemian rhapsody...,Cuba
349928,negative,mileycyrus your not coming to philadelphia bu...,Chile
642649,negative,my throat is killing me but i wanna go to the ...,Thailand
291136,negative,time to hate being female again,Japan
1080187,positive,home is where your heat is the sounds,Egypt
609696,negative,my tummy doesnt feel well,India
65907,negative,my ability to send mobile tweets died for some...,New Zealand


In [74]:
# stop_words = set(stopwords.words('english'))
# df['tweet'] = df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [75]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [76]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: x.lower())

stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

stemmer = PorterStemmer()
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [77]:
df.sample(10)

,sentiment,tweet,location
1181583,positive,dannymcfli pleas give shoutout filipino fan lo...,Italy
432559,negative,studi ccnp realli need slp,Chile
1570901,positive,rappublicist gotchaaaaaa lol jst arent use,Egypt
499698,negative,doesnt want go orient tomorrow whateverrrrr,Ethiopia
576049,negative,tokyobunni never uk never gd artvinyl show arg...,Jamaica
1277546,positive,im look twitter homepag see rockitman theeconz...,Kenya
1580802,positive,use translat filipino homework dont love googl,Jamaica
1148331,positive,go bed umpir softbal game,Mexico
693473,negative,beckyhop miss ppp,Antarctica
656253,negative,applewe regalbeagl probabl fun yumiteele,Argentina
